# Clean

This notebook loads your assembled data and prepares it for modeling.

The final output is a `sup` that is fully ready for modeling and analysis.

Here's what happens in this notebook:
- We fill gaps in the data using reasonable assumptions
- We prepare the data for further analysis by marking clusters
- We process sales validity information
- We run our own sales scrutiny heuristic to make sure we only use trustworthy sales that reflect market value

These operations are necessary for modeling, but nevertheless inject a certain amount of subjectivity into the model, which is why we keep the results of the **Clean** notebook separate from those produced in the **Assemble** notebook.

In [1]:
# Change these as desired

# The slug of the locality you are currently working on
locality = "us-md-baltimorecity"

# Whether to print out a lot of stuff (can help with debugging) or stay mostly quiet
verbose = True

# Clear previous state for this notebook and start fresh
clear_checkpoints = True

# 1. Basic setup

In [2]:
import init_notebooks
init_notebooks.setup_environment()

Environment setup completed.


In [3]:
# import a bunch of stuff
from openavmkit.pipeline import (
    init_notebook,
    from_checkpoint,
    delete_checkpoints,
    write_checkpoint,
    read_pickle,
    load_settings,
    examine_sup,
    fill_unknown_values_sup,
    process_sales,
    mark_ss_ids_per_model_group_sup,
    mark_horizontal_equity_clusters_per_model_group_sup,
    run_sales_scrutiny_per_model_group_sup,
    write_notebook_output_sup
)

In [4]:
init_notebook(locality)

locality = us-md-baltimorecity
base path = /Users/gregmiller/Documents/openavmkit/openavmkit/notebooks/pipeline
current path = /Users/gregmiller/Documents/openavmkit/openavmkit/notebooks/pipeline/data/us-md-baltimorecity


In [5]:
if clear_checkpoints:
    delete_checkpoints("2-clean")

# 2. Load data

In [6]:
settings = load_settings()

In [7]:
# load the data
sales_univ_pair = read_pickle("out/1-assemble-sup")

In [8]:
examine_sup(sales_univ_pair, settings)


EXAMINING UNIVERSE...

            FIELD                 TYPE     NON-ZERO    %    NON-NULL    %                    UNIQUE                 
============================== ========== ========== ===== ========== ===== ========================================
             LAND             
============================== ========== ========== ===== ========== ===== ========================================
------------------------------ ---------- ---------- ----- ---------- ----- ----------------------------------------
           NUMERIC            
------------------------------ ---------- ---------- ----- ---------- ----- ----------------------------------------
assr_land_value                 Float64      219,315   93%    236,505  100%                                         
exempt_land_value               Float64       16,452    7%    236,471  100%                                         
geom_aspect_ratio               float64      236,505  100%    236,505  100%                    

# 3. Fill unknowns

Modeling functions are unable to process null data, so you need to fill them in somehow.   
The goal is to **eliminate all gaps in your data,** at least for fields you intend to turn into modeling variables.

Consult the documentation for more details and best practices on filling unknown values.

In [9]:
# Fill holes in the data with sensible defaults
sales_univ_pair = fill_unknown_values_sup(sales_univ_pair, settings)

In [10]:
# Look at the data and make sure it makes sense
examine_sup(sales_univ_pair, settings)


EXAMINING UNIVERSE...

            FIELD                 TYPE     NON-ZERO    %    NON-NULL    %                    UNIQUE                 
============================== ========== ========== ===== ========== ===== ========================================
             LAND             
============================== ========== ========== ===== ========== ===== ========================================
------------------------------ ---------- ---------- ----- ---------- ----- ----------------------------------------
           NUMERIC            
------------------------------ ---------- ---------- ----- ---------- ----- ----------------------------------------
assr_land_value                 Float64      219,315   93%    236,505  100%                                         
exempt_land_value               Float64       16,452    7%    236,471  100%                                         
geom_aspect_ratio               float64      236,505  100%    236,505  100%                    

# 4. Clustering

We cluster all similar properties and give each cluster a unique ID.  
Later, we'll use these ID's whenever we want to run a horizontal equity study.


In [11]:
settings = load_settings()
sales_univ_pair = from_checkpoint("2-clean-00-horizontal-equity", mark_horizontal_equity_clusters_per_model_group_sup,
    {
        "sup": sales_univ_pair,
        "settings": settings,
        "verbose": verbose
    }
)


Marking horizontal equity clusters...
Processing model group: condos
--> crunching on location, 72 clusters
--> crunching on is_vacant, 77 clusters
--> crunching on is_vacant, 77 clusters
----> 0/77, 0% clustering on SANDTOWN-WINCHESTER_False, field = is_vacant, size = 81
--> crunching on land_area_sqft, 77 clusters
----> 0/77, 0% clustering on SANDTOWN-WINCHESTER_False_0, field = land_area_sqft, size = 81
--> crunching on bldg_area_finished_sqft, 131 clusters
--> crunching on bldg_quality_num, 282 clusters
--> crunching on bldg_condition_num, 282 clusters
--> crunching on bldg_age_years, 282 clusters
Processing model group: apartments
--> crunching on location, 220 clusters
--> crunching on is_vacant, 314 clusters
--> crunching on is_vacant, 314 clusters
----> 0/314, 0% clustering on SANDTOWN-WINCHESTER_False, field = is_vacant, size = 26
--> crunching on land_area_sqft, 314 clusters
--> crunching on bldg_area_finished_sqft, 338 clusters
--> crunching on bldg_quality_num, 342 cluster

In [12]:
sales_univ_pair.universe["bldg_age_years"].value_counts()

bldg_age_years
0.0      57908
105.0    41114
110.0     5974
100.0     5179
85.0      4874
         ...  
217.0        1
221.0        1
219.0        1
189.0        1
197.0        1
Name: count, Length: 216, dtype: Int64

## 5. Process sales

We process sales validity information to set all the right codes for later use.  
We calculate time trends for sales over time to generate time-adjusted sale prices.

In [13]:
sales_univ_pair = from_checkpoint("2-clean-01-process_sales", process_sales,
    {
        "sup": sales_univ_pair,
        "settings": settings,
        "verbose": verbose
    }
)

Before univ merge len = 53837
After univ merge len = 53837
Using 53837 sales...
--> 512 vacant sales
--> 53325 improved sales
--> 53205 valid for ratio study
--> 512 valid for land ratio study
len before hydrate = 53837
len after hydrate = 53837
len after enrich = 53837
Number of parcels in universe with coordinates: 236505 / 236505
Number of parcels in universe with coordinates: 236505 / 236505
Number of parcels in universe with coordinates: 236505 / 236505
Number of parcels in universe with coordinates: 236505 / 236505


## 6. Scrutinize sales

We cluster all sales of similar properties in similar locations.  
We flag individual sales that are anomalously high or low for their local cluster.  
This helps us catch potentially invalid sales that slipped by the assessor's notice.  

In [14]:
sales_univ_pair = from_checkpoint("2-clean-02-mark-ss_ids", mark_ss_ids_per_model_group_sup,
    {
        "sup": sales_univ_pair,
        "settings": settings,
        "verbose": verbose,
    }
)

--> crunching on location, 53 clusters
--> crunching on is_vacant, 56 clusters
--> crunching on land_area_sqft, 56 clusters
--> crunching on bldg_area_finished_sqft, 111 clusters
--> crunching on bldg_quality_num, 254 clusters
--> crunching on ['bldg_effective_age_years', 'bldg_age_years'], 254 clusters
--> crunching on bldg_condition_num, 259 clusters
--> crunching on location, 114 clusters
--> crunching on is_vacant, 121 clusters
--> crunching on land_area_sqft, 121 clusters
--> crunching on bldg_area_finished_sqft, 121 clusters
--> crunching on bldg_quality_num, 121 clusters
--> crunching on ['bldg_effective_age_years', 'bldg_age_years'], 121 clusters
--> crunching on bldg_condition_num, 121 clusters
--> crunching on location, 85 clusters
--> crunching on is_vacant, 99 clusters
--> crunching on land_area_sqft, 99 clusters
--> crunching on bldg_area_finished_sqft, 107 clusters
--> crunching on bldg_quality_num, 107 clusters
--> crunching on ['bldg_effective_age_years', 'bldg_age_year

In [15]:
sales_univ_pair = from_checkpoint("2-clean-03-sales-scrutiny", run_sales_scrutiny_per_model_group_sup,
    {
        "sup": sales_univ_pair,
        "settings": settings,
        "verbose": verbose
    }
)

/Users/gregmiller/Documents/openavmkit/openavmkit/.venv/lib/python3.12/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/Users/gregmiller/Documents/openavmkit/openavmkit/.venv/lib/python3.12/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


# 7. Write out results

In [16]:
write_notebook_output_sup(sales_univ_pair, "2-clean")

Results written to:
...out/2-clean-sup.pickle
...out/look/2-clean-universe.parquet
...out/look/2-clean-sales.parquet
...out/look/2-clean-sales-hydrated.parquet


# 8. Look at it on a map!
- Take the files output in the previous step and put them in a map viewer like QGIS, ArcGIS, or Felt
- Look at them with your eyeballs
- Make sure the data looks correct
- If not, go back and fix it!
- Don't proceed to the next step until everything looks right